In [2]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time
import random

## Loading Training/Testing/Validation Data

In [3]:
dataset = "PhonemeSpectra"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]
print(f"Total Instances: {n_instances}")

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = np.asarray(y_train_load['arr_0'], dtype = np.uint64)
y_test = np.asarray(y_test_load['arr_0'], dtype = np.uint64)

y_all = np.concatenate((y_train, y_test), axis = 0)

n_validation = int(0.1*n_instances)
print(f"Validation Instances: {n_validation}")

ind_validation = random.sample(range(0, n_instances), n_validation)
x_valid = x_all[ind_validation, :, :]
y_valid = y_all[ind_validation]

x_all = np.delete(x_all, ind_validation, axis = 0)
y_all = np.delete(y_all, ind_validation, axis = 0)

print(f"x_all: {x_all.shape} - y_all: {y_all.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

Total Instances: 6668
Validation Instances: 666
x_all: (6002, 217, 11) - y_all: (6002,)
x_valid: (666, 217, 11) - y_valid: (666,)


In [5]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_train.shape[1]
n_features = x_train.shape[-1]

model = Sequential()

model.add(RNN(OrthogonalCell(units = 212,
                             order = 256,
                             variant = 'ct_laguerre',
                             dt = 1,
                            input_dims = n_features), 
              input_shape = (length, n_features),
             return_sequences = False))
model.add(Dense(to_categorical(y_train).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 212)               172961    
_________________________________________________________________
dense_1 (Dense)              (None, 39)                8307      
Total params: 181,268
Trainable params: 112,894
Non-trainable params: 68,374
_________________________________________________________________


In [6]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-laguerre.hdf5'))

callbacks = [
    ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
    EarlyStopping(monitor = 'val_loss', patience = 50, mode = 'min')]

t = time.time()

result = model.fit(x_train, 
                   to_categorical(y_train),
                   epochs = 2000, 
                   batch_size = 100, 
                   validation_data = (x_valid, to_categorical(y_valid)), 
                   callbacks = callbacks)

print(f"Training time: {time.time() - t} s")

df_results = pd.DataFrame(result.history)
df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'laguerre_results.csv')))

Train on 3315 samples, validate on 666 samples
Epoch 1/2000
3300/3315 [============================>.] - ETA: 0s - loss: 3.7843 - acc: 0.0364
Epoch 00001: val_loss improved from inf to 3.47894, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PhonemeSpectra/PhonemeSpectra-laguerre.hdf5
3315/3315 [==============================] - 4s 1ms/sample - loss: 3.7846 - acc: 0.0365 - val_loss: 3.4789 - val_acc: 0.1006
Epoch 2/2000
3300/3315 [============================>.] - ETA: 0s - loss: 3.3662 - acc: 0.1224
Epoch 00002: val_loss improved from 3.47894 to 3.36566, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PhonemeSpectra/PhonemeSpectra-laguerre.hdf5
3315/3315 [==============================] - 4s 1ms/sample - loss: 3.3668 - acc: 0.1228 - val_loss: 3.3657 - val_acc: 0.1396
Epoch 3/2000
3300/3315 [============================>.] - ETA: 0s - loss: 3.1933 - acc: 0.1770
Epoch 00003: val_loss improved f

KeyboardInterrupt: 

In [20]:
saved_epoch = np.argmin(result.history['val_loss'])

print(f"Epoch {saved_epoch}: {result.history['val_acc'][saved_epoch]} val_acc")

model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

Epoch 36: 0.9326660633087158 val_acc
2199/2199 [==============================] - 0s 58us/sample - loss: 0.1928 - acc: 0.9409


[0.19282045773399478, 0.9408822]